**Problem Statement**: This notebook provides the details about Machine learning predictive model for the problem “Bank loan default risk”.

# **1. importing required libraries and reading the data**


In [ ]:
# Code to mount google drive   (loading the data from your google drive)
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold      # for Feature selection
from sklearn.feature_selection import mutual_info_classif    # Feature selection - mutual information
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
from sklearn.ensemble import IsolationForest



# **Reading data from pickle file**

In [ ]:
import pickle
import joblib 
import pandas as pd

# joblib.dump(FinalXGboostModel, r'/gdrive/MyDrive/PGD_project/FinalXGboostModel.pkl') 
# model = joblib.load(r'/gdrive/MyDrive/PGD_project/FinalXGboostModel.pkl') 
# model.predict(X_test)

DataForML_t=pd.read_pickle(r'/gdrive/MyDrive/PGD_project/DataForML_t.pkl')

In [ ]:
DataForML_t.head()

,TARGET,AGE,AMT_CREDIT,AMT_INCOME_TOTAL,CNT_FAM_MEMBERS,YEARS_EMPLOYED,AMT_ANNUITY,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_6,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,EXT_SOURCE_2,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,CODE_GENDER,FLAG_OWN_CAR,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,NAME_EDUCATION_TYPE,NAME_CONTRACT_TYPE,FLAG_OWN_REALTY,NAME_TYPE_SUITE,ORGANIZATION_TYPE
0,1,25.0,406597.5,202500.0,1.0,1.0,24700.5,1,0,1,1,0,2,2,0,0,0.262949,1,3648.0,2120,2,2.0,Male,N,Working,Single / not married,House / apartment,Secondary / secondary special,Cash loans,Y,Unaccompanied,Business Entity Type 3
1,0,45.0,1293502.5,270000.0,2.0,3.0,35698.5,1,0,1,1,0,1,1,0,0,0.622246,0,1186.0,291,1,0.0,Female,N,State servant,Married,House / apartment,Higher education,Cash loans,N,Family,School
2,0,52.0,135000.0,67500.0,1.0,0.0,6750.0,1,1,1,0,0,2,2,0,0,0.555912,0,4260.0,2531,0,0.0,Male,Y,Working,Single / not married,House / apartment,Secondary / secondary special,Revolving loans,Y,Unaccompanied,Government
3,0,52.0,312682.5,135000.0,2.0,8.0,29686.5,1,0,0,1,0,2,2,0,0,0.650442,0,9833.0,2437,2,0.0,Female,N,Working,Civil marriage,House / apartment,Secondary / secondary special,Cash loans,Y,Unaccompanied,Business Entity Type 3
4,0,54.0,513000.0,121500.0,1.0,8.0,21865.5,1,0,0,0,0,2,2,0,1,0.322738,0,4311.0,3458,0,0.0,Male,N,Working,Single / not married,House / apartment,Secondary / secondary special,Cash loans,Y,Unaccompanied,Religion


In [ ]:
DataForML_t['CODE_GENDER'].replace({'Male':1, 'Female':0}, inplace=True)
DataForML_t['FLAG_OWN_CAR'].replace({'Y':1, 'N':0}, inplace=True)
DataForML_t['NAME_CONTRACT_TYPE'].replace({'Revolving loans':1, 'Cash loans':0}, inplace=True)
DataForML_t['FLAG_OWN_REALTY'].replace({'Y':1, 'N':0}, inplace=True)

In [ ]:
# Treating all the nominal variables at once using dummy variables
DataForML_Numeric=pd.get_dummies(DataForML_t)
DataForML_Numeric.head()

,TARGET,AGE,AMT_CREDIT,AMT_INCOME_TOTAL,CNT_FAM_MEMBERS,YEARS_EMPLOYED,AMT_ANNUITY,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_6,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,EXT_SOURCE_2,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,CODE_GENDER,FLAG_OWN_CAR,NAME_CONTRACT_TYPE,FLAG_OWN_REALTY,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,...,ORGANIZATION_TYPE_Industry: type 13,ORGANIZATION_TYPE_Industry: type 2,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 4,ORGANIZATION_TYPE_Industry: type 5,ORGANIZATION_TYPE_Industry: type 6,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA
0,1,25.0,406597.5,202500.0,1.0,1.0,24700.5,1,0,1,1,0,2,2,0,0,0.262949,1,3648.0,2120,2,2.0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,45.0,1293502.5,270000.0,2.0,3.0,35698.5,1,0,1,1,0,1,1,0,0,0.622246,0,1186.0,291,1,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,52.0,135000.0,67500.0,1.0,0.0,6750.0,1,1,1,0,0,2,2,0,0,0.555912,0,4260.0,2531,0,0.0,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,52.0,312682.5,135000.0,2.0,8.0,29686.5,1,0,0,1,0,2,2,0,0,0.650442,0,9833.0,2437,2,0.0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,54.0,513000.0,121500.0,1.0,8.0,21865.5,1,0,0,0,0,2,2,0,1,0.322738,0,4311.0,3458,0,0.0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(DataForML_Numeric.shape)

(306486, 116)


# **Machine Learning**



In [ ]:
# Printing all the column names for our reference
print(list(DataForML_Numeric.columns))

['TARGET', 'AGE', 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'CNT_FAM_MEMBERS', 'YEARS_EMPLOYED', 'AMT_ANNUITY', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_6', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'EXT_SOURCE_2', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE_Businessman', 'NAME_INCOME_TYPE_Commercial associate', 'NAME_INCOME_TYPE_Maternity leave', 'NAME_INCOME_TYPE_Pensioner', 'NAME_INCOME_TYPE_State servant', 'NAME_INCOME_TYPE_Student', 'NAME_INCOME_TYPE_Unemployed', 'NAME_INCOME_TYPE_Working', 'NAME_FAMILY_STATUS_Civil marriage', 'NAME_FAMILY_STATUS_Married', 'NAME_FAMILY_STATUS_Separated', 'NAME_FAMILY_STATUS_Single / not married', 'NAME_FAMILY_STATUS_Unknown', 'NAME_FAMILY_STATUS_Widow', 'NAME_HOUSING_TYPE_Co-op a

In [ ]:
# Separate Target Variable and Predictor Variables
TargetVariable='TARGET'
Predictors=[ 'AGE', 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'CNT_FAM_MEMBERS', 'YEARS_EMPLOYED', 'AMT_ANNUITY', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 
             'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_6', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 
             'EXT_SOURCE_2', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 
             'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',  'FLAG_OWN_REALTY',
             
             'NAME_TYPE_SUITE_Unaccompanied', 'NAME_TYPE_SUITE_Family', 'NAME_TYPE_SUITE_Spouse, partner', 'NAME_TYPE_SUITE_Children', 
             'NAME_TYPE_SUITE_Other_B', 'NAME_TYPE_SUITE_Other_A', 'NAME_TYPE_SUITE_Group of people',
             
             'NAME_EDUCATION_TYPE_Secondary / secondary special', 'NAME_EDUCATION_TYPE_Higher education', 'NAME_EDUCATION_TYPE_Incomplete higher',
             'NAME_EDUCATION_TYPE_Lower secondary', 'NAME_EDUCATION_TYPE_Academic degree',  
               
             'NAME_FAMILY_STATUS_Married', 'NAME_FAMILY_STATUS_Single / not married', 'NAME_FAMILY_STATUS_Civil marriage', 
             'NAME_FAMILY_STATUS_Separated', 'NAME_FAMILY_STATUS_Widow', 'NAME_FAMILY_STATUS_Unknown',     

             'NAME_HOUSING_TYPE_House / apartment', 'NAME_HOUSING_TYPE_With parents', 'NAME_HOUSING_TYPE_Municipal apartment',  
             'NAME_HOUSING_TYPE_Rented apartment', 'NAME_HOUSING_TYPE_Office apartment', 'NAME_HOUSING_TYPE_Co-op apartment',         

             'ORGANIZATION_TYPE_Business Entity Type 3', 'ORGANIZATION_TYPE_XNA', 'ORGANIZATION_TYPE_Self-employed', 'ORGANIZATION_TYPE_Other',
             'ORGANIZATION_TYPE_Medicine', 'ORGANIZATION_TYPE_Business Entity Type 2', 'ORGANIZATION_TYPE_Government', 'ORGANIZATION_TYPE_School',
             'ORGANIZATION_TYPE_Trade: type 7', 'ORGANIZATION_TYPE_Kindergarten', 'ORGANIZATION_TYPE_Construction', 'ORGANIZATION_TYPE_Business Entity Type 1',
             'ORGANIZATION_TYPE_Transport: type 4', 'ORGANIZATION_TYPE_Trade: type 3', 'ORGANIZATION_TYPE_Industry: type 9', 'ORGANIZATION_TYPE_Industry: type 3',
             'ORGANIZATION_TYPE_Security', 'ORGANIZATION_TYPE_Housing', 'ORGANIZATION_TYPE_Industry: type 11', 'ORGANIZATION_TYPE_Military',
             'ORGANIZATION_TYPE_Bank', 'ORGANIZATION_TYPE_Agriculture', 'ORGANIZATION_TYPE_Police', 'ORGANIZATION_TYPE_Transport: type 2',
             'ORGANIZATION_TYPE_Postal', 'ORGANIZATION_TYPE_Security Ministries', 'ORGANIZATION_TYPE_Trade: type 2', 'ORGANIZATION_TYPE_Restaurant',
             'ORGANIZATION_TYPE_Services', 'ORGANIZATION_TYPE_University', 'ORGANIZATION_TYPE_Industry: type 7', 'ORGANIZATION_TYPE_Transport: type 3',
             'ORGANIZATION_TYPE_Industry: type 1', 'ORGANIZATION_TYPE_Hotel', 'ORGANIZATION_TYPE_Electricity', 'ORGANIZATION_TYPE_Industry: type 4',
             'ORGANIZATION_TYPE_Trade: type 6', 'ORGANIZATION_TYPE_Industry: type 5', 'ORGANIZATION_TYPE_Insurance', 'ORGANIZATION_TYPE_Telecom',
             'ORGANIZATION_TYPE_Emergency', 'ORGANIZATION_TYPE_Industry: type 2', 'ORGANIZATION_TYPE_Advertising', 'ORGANIZATION_TYPE_Realtor',
             'ORGANIZATION_TYPE_Culture', 'ORGANIZATION_TYPE_Industry: type 12', 'ORGANIZATION_TYPE_Trade: type 1',
             'ORGANIZATION_TYPE_Mobile', 'ORGANIZATION_TYPE_Legal Services', 'ORGANIZATION_TYPE_Cleaning', 'ORGANIZATION_TYPE_Transport: type 1', 
             'ORGANIZATION_TYPE_Industry: type 6', 'ORGANIZATION_TYPE_Industry: type 10', 'ORGANIZATION_TYPE_Religion', 'ORGANIZATION_TYPE_Industry: type 13',
             'ORGANIZATION_TYPE_Trade: type 4', 'ORGANIZATION_TYPE_Trade: type 5', 'ORGANIZATION_TYPE_Industry: type 8',    
                  
             'NAME_INCOME_TYPE_Working', 'NAME_INCOME_TYPE_Commercial associate', 'NAME_INCOME_TYPE_Pensioner', 'NAME_INCOME_TYPE_State servant', 
             'NAME_INCOME_TYPE_Unemployed', 'NAME_INCOME_TYPE_Student', 'NAME_INCOME_TYPE_Businessman', 'NAME_INCOME_TYPE_Maternity leave']

X_ml=DataForML_Numeric[Predictors].values
y_ml=DataForML_Numeric[TargetVariable].values

# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(X_ml, y_ml, test_size=0.3, random_state=428)


# **Standardization/Normalization of data**

In [ ]:
### Sandardization of data ###
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Choose either standardization or Normalization
# On this data Min Max Normalization produced better results

# Choose between standardization and MinMAx normalization
#PredictorScaler=StandardScaler()
PredictorScaler=MinMaxScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X_ml)

# Generating the standardized values of X
X_ml=PredictorScalerFit.transform(X_ml)

# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(X_ml, y_ml, test_size=0.3, random_state=42)

In [ ]:
# Sanity check for the sampled data
print(X_train_ml.shape)
print(y_train_ml.shape)
print(X_test_ml.shape)
print(y_test_ml.shape)

(214540, 115)
(214540,)
(91946, 115)
(91946,)


In [ ]:
# Get the default and non-default

# default_1     = y_test[y_test['TARGET']==1]
# non_default_1 = y_test[y_test['TARGET']==0]

default_1     = y_test_ml[y_test_ml==1]
non_default_1 = y_test_ml[y_test_ml==0]

print(default_1.shape,non_default_1.shape)

(7437,) (84509,)


# **Oversampling**

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
os = RandomOverSampler(0.75)
X_train_mlos, y_train_mlos = os.fit_resample(X_train_ml, y_train_ml)

print("The number of classes before fit {}".format(Counter(y_train_ml)))
print("The number of classes after fit {}".format(Counter(y_train_mlos)))

/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:591: FutureWarning: Pass sampling_strategy=0.5 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning,


The number of classes before fit Counter({0: 197188, 1: 17352})
The number of classes after fit Counter({0: 197188, 1: 98594})


In [ ]:
X_train_mlos.shape

(295782, 115)

In [ ]:
X_train_ml.shape

(214540, 115)

# **Anamoly Detection and Removal**

In [ ]:
# identify outliers in the training dataset
iso = IsolationForest(contamination=0.1)
y_pred = iso.fit_predict(X_train_mlos)

In [ ]:
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
# function to get unique values
def unique(list1):
    y_pred_unique = np.array(list1)
    print(np.unique(y_pred_unique))

In [ ]:
print("the unique values from 1st list is")
unique(y_pred)

the unique values from 1st list is
[-1  1]


In [ ]:
# y_train_mlos.head()

In [ ]:
# select all rows that are not outliers
mask = y_pred != -1
X_train_iso, y_train_iso = X_train_mlos[mask], y_train_mlos[mask]

In [ ]:
X_train_mlos.shape, y_train_mlos.shape

((295782, 115), (295782,))

In [ ]:
X_train_iso.shape, y_train_iso.shape

((266203, 115), (266203,))

In [ ]:
print("The number of classes after outlier removal {}".format(Counter(y_train_iso)))

The number of classes after outlier removal Counter({0: 177331, 1: 88872})


# **Neural Network**

Building a multi-layer neaural network model

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train_mlann = to_categorical(y_train_mlos)
y_test_mlann = to_categorical(y_test_ml)

In [ ]:
# sequential model
ann_model = keras.Sequential()

# adding dense layer
ann_model.add(Dense(250, input_dim=115, kernel_initializer='normal', activation='relu'))
ann_model.add(Dropout(0.3))
ann_model.add(Dense(500, activation='relu'))
ann_model.add(Dropout(0.3))
ann_model.add(Dense(500, activation='relu'))
ann_model.add(Dropout(0.3))
ann_model.add(Dense(500, activation='relu'))
ann_model.add(Dropout(0.4))
ann_model.add(Dense(250, activation='linear'))
ann_model.add(Dropout(0.4))

# adding dense layer with softmax activation/output layer
ann_model.add(Dense(1, activation='sigmoid'))
# ann_model.add(Dense(1, activation='softmax'))
ann_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 250)               29000     
                                                                 
 dropout_20 (Dropout)        (None, 250)               0         
                                                                 
 dense_28 (Dense)            (None, 500)               125500    
                                                                 
 dropout_21 (Dropout)        (None, 500)               0         
                                                                 
 dense_29 (Dense)            (None, 500)               250500    
                                                                 
 dropout_22 (Dropout)        (None, 500)               0         
                                                                 
 dense_30 (Dense)            (None, 500)              

In [ ]:
# custom functions for f1, precision and recall

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# ann_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1_m]) # metrics=['accuracy']
ann_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1_m]) # metrics=['accuracy']

In [ ]:
X_train_mlos.shape, X_test_ml.shape, y_train_mlos.shape, y_test_ml.shape

((345079, 115), (91946, 115), (345079,), (91946,))

In [ ]:
X_train_mlos.shape, X_test_ml.shape, y_train_mlann.shape, y_test_mlann.shape

((345079, 115), (91946, 115), (345079, 2), (91946, 2))

In [ ]:
# history = ann_model.fit(X_train_mlos, y_train_mlos, validation_data = (X_test_ml, y_test_ml), epochs = 50)
history = ann_model.fit(X_train_mlos, y_train_mlos, epochs=20, validation_split=0.2, verbose=1)


Epoch 1/20
8627/8627 [==============================] - 118s 14ms/step - loss: 0.5578 - f1_m: 0.2020 - val_loss: 1.1685 - val_f1_m: 0.2884
Epoch 2/20
8627/8627 [==============================] - 113s 13ms/step - loss: 0.5503 - f1_m: 0.2265 - val_loss: 1.1477 - val_f1_m: 0.0937
Epoch 3/20
8627/8627 [==============================] - 114s 13ms/step - loss: 0.5462 - f1_m: 0.2451 - val_loss: 1.2120 - val_f1_m: 0.0509
Epoch 4/20
8627/8627 [==============================] - 114s 13ms/step - loss: 0.5421 - f1_m: 0.2602 - val_loss: 1.1522 - val_f1_m: 0.1823
Epoch 5/20
8627/8627 [==============================] - 113s 13ms/step - loss: 0.5388 - f1_m: 0.2728 - val_loss: 1.2814 - val_f1_m: 0.1282
Epoch 6/20
8627/8627 [==============================] - 117s 14ms/step - loss: 0.5362 - f1_m: 0.2760 - val_loss: 1.1325 - val_f1_m: 0.2077
Epoch 7/20
8627/8627 [==============================] - 117s 14ms/step - loss: 0.5331 - f1_m: 0.2901 - val_loss: 1.1366 - val_f1_m: 0.1121
Epoch 8/20
8627/8627 [=====

Evaluating model performance

In [ ]:
predictions = ann_model.predict(X_test_ml)
predict = []

for i in predictions:
    predict.append(np.argmax(i))

In [ ]:
print(predict)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
# Getting predictions for the test set
prediction_1 = (model.predict(X_test_ml) > 0.5).astype("int32")
# Getting accuracy score
from sklearn.metrics import accuracy_score
print("Accuracy_Score : {}".format(accuracy_score(prediction_1, y_test_ml) * 100))

Accuracy_Score : 72.47623605159549


In [ ]:
print(prediction_1)

[[0]
 [0]
 [1]
 ...
 [1]
 [1]
 [0]]


In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction_1))
print(metrics.confusion_matrix(y_test_ml, prediction_1))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test_ml, prediction_1, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

              precision    recall  f1-score   support

           0       0.95      0.74      0.83     84509
           1       0.15      0.53      0.24      7437

    accuracy                           0.72     91946
   macro avg       0.55      0.64      0.54     91946
weighted avg       0.88      0.72      0.78     91946

[[62667 21842]
 [ 3465  3972]]
Accuracy of the model on Testing Sample Data: 0.78


In [ ]:
# Getting predictions for the test set
prediction_all = (model.predict(X_ml) > 0.5).astype("int32")
# Getting accuracy score
from sklearn.metrics import accuracy_score
print("Accuracy_Score : {}".format(accuracy_score(prediction_all, y_ml) * 100))

Accuracy_Score : 72.77004496127066


In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_ml, prediction_all))
print(metrics.confusion_matrix(y_ml, prediction_all))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_ml, prediction_all, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

              precision    recall  f1-score   support

           0       0.95      0.74      0.83    281697
           1       0.16      0.55      0.25     24789

    accuracy                           0.73    306486
   macro avg       0.55      0.65      0.54    306486
weighted avg       0.89      0.73      0.79    306486

[[209324  72373]
 [ 11083  13706]]
Accuracy of the model on Testing Sample Data: 0.79


# **Neural Network 2**

In [ ]:
# Creating a skeleton of model.
from keras.models import Sequential
# Defining layers
from keras.layers import Input, Dense
from tensorflow.keras.layers import InputLayer


In [ ]:
# Input neurons
input_neurons = X_train_mlos.shape[1]
# Output neurons (Since it is binary classification)
output_neurons = 1
# Defining hidden layers & neurons in each layersnumber_of_hidden_layers = 2
neuron_hidden_layer_1         = 10
neuron_hidden_layer_2         = 5
# Defining the architecture of the model
model = Sequential()
model.add(InputLayer(input_shape=(input_neurons)))
model.add(Dense(units=neuron_hidden_layer_1, activation='relu'))
model.add(Dense(units=neuron_hidden_layer_2, activation='relu'))
model.add(Dense(units=output_neurons, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
model_history = model.fit(X_train_mlos, y_train_mlos, validation_data = (X_test_ml, y_test_ml), epochs = 20)

Epoch 1/20
10784/10784 [==============================] - 34s 3ms/step - loss: 0.6203 - accuracy: 0.6561 - val_loss: 0.5272 - val_accuracy: 0.7600
Epoch 2/20
10784/10784 [==============================] - 29s 3ms/step - loss: 0.6178 - accuracy: 0.6587 - val_loss: 0.5027 - val_accuracy: 0.7839
Epoch 3/20
10784/10784 [==============================] - 21s 2ms/step - loss: 0.6160 - accuracy: 0.6611 - val_loss: 0.5180 - val_accuracy: 0.7615
Epoch 4/20
10784/10784 [==============================] - 23s 2ms/step - loss: 0.6149 - accuracy: 0.6617 - val_loss: 0.4977 - val_accuracy: 0.7787
Epoch 5/20
10784/10784 [==============================] - 23s 2ms/step - loss: 0.6139 - accuracy: 0.6626 - val_loss: 0.5454 - val_accuracy: 0.7327
Epoch 6/20
10784/10784 [==============================] - 21s 2ms/step - loss: 0.6129 - accuracy: 0.6639 - val_loss: 0.5581 - val_accuracy: 0.7208
Epoch 7/20
10784/10784 [==============================] - 22s 2ms/step - loss: 0.6123 - accuracy: 0.6646 - val_loss: 0

In [ ]:
# Getting predictions for the test set
prediction = (model.predict(X_test_ml) > 0.5).astype("int32")
# Getting accuracy score
from sklearn.metrics import accuracy_score
print("Accuracy_Score : {}".format(accuracy_score(prediction, y_test_ml) * 100))

Accuracy_Score : 72.47623605159549


In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction))
print(metrics.confusion_matrix(y_test_ml, prediction))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test_ml, prediction, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

              precision    recall  f1-score   support

           0       0.95      0.74      0.83     84509
           1       0.15      0.53      0.24      7437

    accuracy                           0.72     91946
   macro avg       0.55      0.64      0.54     91946
weighted avg       0.88      0.72      0.78     91946

[[62667 21842]
 [ 3465  3972]]
Accuracy of the model on Testing Sample Data: 0.78
